In [1]:
from pathlib import Path
from pprint import pprint

DATA_DIR = Path("../data")
SNAPSHOTS_DIR = DATA_DIR / "platform-docs-snapshots"
VERSIONS_DIR = DATA_DIR / "platform-docs-versions"

# hparams
chunk_size = 1024
chunk_overlap = chunk_size // 10
embedder_name = "BAAI/bge-small-en-v1.5" # https://huggingface.co/spaces/mteb/leaderboard

# Pre-processing

In [2]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader, DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

docs = DirectoryLoader(VERSIONS_DIR, glob="[!.]*/[!.]*.md", loader_cls=TextLoader).load()
docs = [d for d in docs if Path(d.metadata['source']) != VERSIONS_DIR / "README.md"]

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
docs_split = text_splitter.split_documents(docs)

# Build index

In [5]:
import uuid
from tqdm.notebook import tqdm
import chromadb
from chromadb.utils import embedding_functions
import re

# Chroma uses all-MiniLM-L6-v2 by default
chroma_client = chromadb.PersistentClient()
 
def format_model_name(name):
    # chromaDB only allows these characters
    return re.sub(r'[^a-zA-Z0-9_-]', '_', name)

collection_name = f"DSA_{format_model_name(embedder_name)}"

try:
    collection = chroma_client.get_collection(collection_name)
except ValueError:
    print("Building collection...")
    embedding_fn = embedding_functions.SentenceTransformerEmbeddingFunction(embedder_name=embedder_name)

    collection = chroma_client.create_collection(name=collection_name, embedding_function=embedding_fn)
    
    text = [d.page_content for d in docs_split]
    metadatas = [d.metadata for d in docs_split]
    ids = [uuid.uuid4().hex for _ in range(len(docs_split))]
    
    collection.add(
        documents=text,
        metadatas=metadatas,
        ids=ids,
    )

In [6]:
results = collection.query(
    query_texts=["How do I get activity logs from the Twitter API?"],
    n_results=5,
)

In [7]:
pprint(results["metadatas"])
results["documents"][0][0]

[[{'source': '../data/platform-docs-versions/X_Twitter-API-Enterprise/Account '
             'Activity API.md'},
  {'source': '../data/platform-docs-versions/X_Twitter-API-Enterprise/Account '
             'Activity API.md'},
  {'source': '../data/platform-docs-versions/X_Twitter-API-Enterprise/Account '
             'Activity API.md'},
  {'source': '../data/platform-docs-versions/X_Twitter-API-Enterprise/Account '
             'Activity API.md'},
  {'source': '../data/platform-docs-versions/X_Twitter-API-Enterprise/Account '
             'Activity API.md'}]]


'####\n\nAuthenticating with the Account Activity API\n\n**Please note**:\xa0\n\nTwitter needs to enable access to the Account Activity API for your developer App before you can start using the API. To this end, make sure to share the App ID that you intend to use for authentication purposes with your account manager or technical support team.\n\nThe [Account Activity API](https://developer.twitter.com/en/docs/twitter-api/enterprise/account-activity-api/overview) consists of a set of endpoints that allow you to create and manage user subscriptions to receive real-time account activities for all of your subscribed accounts through a single connection.'

# Load Generator

### Quantized Mistral 7B, finetuned on code instructions
- https://huggingface.co/TheBloke/OpenHermes-2.5-Mistral-7B-16k-GGUF#provided-files

In [8]:
from ctransformers import AutoModelForCausalLM, AutoConfig
from transformers import AutoTokenizer
from typing import List, Dict

gpu_layers = 0

model_file = "openhermes-2.5-mistral-7b-16k.Q4_K_M.gguf"

llm = AutoModelForCausalLM.from_pretrained(
    "TheBloke/OpenHermes-2.5-Mistral-7B-16k-GGUF",
    model_file="openhermes-2.5-mistral-7b-16k.Q4_K_M.gguf",
    model_type="mistral", 
    gpu_layers=gpu_layers,
    max_new_tokens=4000,
    context_length=16_000,
)

tokenizer = AutoTokenizer.from_pretrained("NurtureAI/OpenHermes-2.5-Mistral-7B-16k")

def format_prompt(messages: List[Dict[str, str]]) -> str:
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# RAG Query

In [9]:
system_message = """
You are a friendly QA bot that answers questions based only on the context you are given.
- Give an  answer only based on the context and with no prior knowledge.
- If you cannot come up with an answer to the user question, answer that you do not know the answer.
- You can be very strict with what you do and don't answer. if the query is completely unrelated to the context you are given,
reply that you do not know the answer.
- include API code with the python requests library if required

------------------
CONTEXT
{context}
"""

In [10]:
def init_chat_log(context: str):
    return [
        {"role": "system", "content": system_message.format(context=context)},
    ]

def generate_answer(chat_log: List[Dict[str, str]], query: str):
    if query:
        chat_log.append({"role": "user", "content": query})
    if chat_log[-1]["role"] != "user":
        raise ValueError("query required")
            
    prompt = format_prompt(chat)
    output = llm(prompt, stop=["<|im_end|>"])
    chat_log.append({"role": "assistant", "content": output})
    return chat_log

from IPython.display import display, Markdown

def print_chat(chat_log):
    for entry in chat_log:
        if entry['role'] != 'system':
            display(Markdown(f"**{entry['role'].capitalize()}:** \n{entry['content']}\n"))

In [13]:
results["metadatas"][0][1]["source"]

'../data/platform-docs-versions/Facebook_Graph-API/User Endpoint.md'

In [24]:
k = 5

query = "Tell me how to work with WhatsApp business accounts using the Facebook Graph API"
results = collection.query(
    query_texts=[query],
    n_results=k,
)

contexts = []
for i in range(k):
    source_i = results["metadatas"][0][i]["source"]
    doc_i = results["documents"][0][i]
    contexts.append(f"CONTEXT {i}\nFILENAME: {source_i}\n{doc_i}")

context = "\n\n".join(contexts)
print(context)

CONTEXT 0
FILENAME: ../data/platform-docs-versions/Facebook_Graph-API/Graph API Overview.md
#### X-Ad-Account-Usage Header Contents

| Key | Value Description |
| --- | --- |
| `acc_id_util_pct` | The percentage of calls made for this ad account before the rate limit is reached. |
| `reset_time_duration` | Time duration (in seconds) it takes to reset the current rate limit to 0. |
| `ads_api_access_tier` | Tiers allows your app to access the Marketing API. By default, apps are in the `development_access` tier. `Standard_access` enables lower rate limiting. To get a higher rate limit and get to the standard tier, you can apply for the "Advanced Access" to the [Ads Management Standard Access](https://developers.facebook.com/docs/marketing-api/overview/authorization#layer-2--access-levels--permissions--and-features) feature. |

#### Total CPU Time

The amount of CPU time the request takes to process. When `total_cputime` reaches 100, calls may be throttled.

#### Total Time

The length of

In [25]:
chat = init_chat_log(context)
chat = generate_answer(chat, query)
print_chat(chat)

**User:** 
Tell me how to work with WhatsApp business accounts using the Facebook Graph API


**Assistant:** 
To work with WhatsApp Business Accounts using the Facebook Graph API, you need to follow these requirements:
- have `whatsapp_business_management`, `whatsapp_messaging`, and `public_profile` permissions
- provide a valid WhatsApp Business Account (WABA) ID
- use a USER ACCESS TOKEN

To access the account information of a WhatsApp Business Account, you can make a GET request to the Graph API endpoint for that specific WABA. The request can be made using cURL or in SDKs such as Android and Objective-C. Here's an example:

cURL:
```
curl -i -X GET \
 "https://graph.facebook.com/LATEST-VERSION/WHATSAPP-BUSINESS-ACCOUNT-ID?access_token=USER-ACCESS-TOKEN"
```

Objective-C SDK:
```java
GraphRequest request = GraphRequest.newGraphPathRequest(accessToken,
        "/WHATSAPP-BUSINESS-ACCOUNT-ID",
        new GraphRequest.Callback() {
            @Override
            public void onCompleted(GraphResponse response) {
                // Insert your code here
            }
        });
request.executeAsync();
```
Please make sure to replace `LATEST-VERSION`, `WHATSAPP-BUSINESS-ACCOUNT-ID`, and `USER-ACCESS-TOKEN` with the actual version, account ID, and access token, respectively. For more details and code samples in other languages, refer to the official [WhatsApp Business API documentation](https://developers.facebook.com/docs/graph-api/reference/whats-app-business-account).



In [26]:
chat = generate_answer(chat, query="and how do I cook pancakes?")
print_chat(chat)

**User:** 
Tell me how to work with WhatsApp business accounts using the Facebook Graph API


**Assistant:** 
To work with WhatsApp Business Accounts using the Facebook Graph API, you need to follow these requirements:
- have `whatsapp_business_management`, `whatsapp_messaging`, and `public_profile` permissions
- provide a valid WhatsApp Business Account (WABA) ID
- use a USER ACCESS TOKEN

To access the account information of a WhatsApp Business Account, you can make a GET request to the Graph API endpoint for that specific WABA. The request can be made using cURL or in SDKs such as Android and Objective-C. Here's an example:

cURL:
```
curl -i -X GET \
 "https://graph.facebook.com/LATEST-VERSION/WHATSAPP-BUSINESS-ACCOUNT-ID?access_token=USER-ACCESS-TOKEN"
```

Objective-C SDK:
```java
GraphRequest request = GraphRequest.newGraphPathRequest(accessToken,
        "/WHATSAPP-BUSINESS-ACCOUNT-ID",
        new GraphRequest.Callback() {
            @Override
            public void onCompleted(GraphResponse response) {
                // Insert your code here
            }
        });
request.executeAsync();
```
Please make sure to replace `LATEST-VERSION`, `WHATSAPP-BUSINESS-ACCOUNT-ID`, and `USER-ACCESS-TOKEN` with the actual version, account ID, and access token, respectively. For more details and code samples in other languages, refer to the official [WhatsApp Business API documentation](https://developers.facebook.com/docs/graph-api/reference/whats-app-business-account).



**User:** 
and how do I cook pancakes?


**Assistant:** 
I don't have any knowledge about cooking recipes. My context is limited to the Facebook Graph API documentation. If you need assistance with cooking, it would be best to consult a culinary resource or search for recipes online.
